<a href="https://colab.research.google.com/github/abishekP101/Pytorch/blob/main/NNmodule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class Model(nn.Module):
    def __init__(self , num_features):
      super().__init__()
      self.linear1 = nn.Linear(num_features , 3)
      self.relu = nn.ReLU()
      self.linear2 = nn.Linear(3 , 1)
      self.sigmoid = nn.Sigmoid()

    def forward(self , features):
      out = self.linear1(features)
      out = self.relu(out)
      out = self.linear2(out)
      out = self.sigmoid(out)
      return out

In [ ]:
features = torch.rand(10 , 5)
model = Model(features.shape[1])
model(features)

tensor([[0.3938],
        [0.3943],
        [0.4118],
        [0.4208],
        [0.4113],
        [0.3938],
        [0.3938],
        [0.4111],
        [0.3938],
        [0.3938]], grad_fn=<SigmoidBackward0>)

## Using Sequential

In [ ]:
class Model(nn.Module):
    def __init__(self , num_features):
      super().__init__()
      self.network = nn.Sequential(
          nn.Linear(num_features, 3),
          nn.ReLU(),
          nn.Linear(3 , 1),
          nn.Sigmoid()
      )


    def forward(self , features):
      out = self.network(features)
      return out

In [ ]:
features = torch.rand(10 , 5)
model = Model(features.shape[1])
model(features)

tensor([[0.4568],
        [0.4468],
        [0.4780],
        [0.4728],
        [0.4771],
        [0.4262],
        [0.4303],
        [0.4976],
        [0.4117],
        [0.4535]], grad_fn=<SigmoidBackward0>)

## Improving the training pipeline

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


df  = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
df.drop(columns=['Unnamed: 32' , 'id'] , inplace=True)

In [ ]:
X = df.drop(columns=['diagnosis'])
y = df['diagnosis']

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
label = LabelEncoder()
y_train = label.fit_transform(y_train)
y_test = label.transform(y_test)

In [ ]:
X_train_tensor = torch.tensor(X_train , dtype=torch.float32)
X_test_tensor = torch.tensor(X_test , dtype=torch.float32)
y_train_tensor = torch.tensor(y_train , dtype=torch.float32)
y_test_tensor = torch.tensor(y_test , dtype=torch.float32)

In [ ]:
import torch
import torch.nn as nn

class MySimpleNN(nn.Module):
  def __init__(self , features):
    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(features , 3),
        nn.ReLU(),
        nn.Linear(3 , 1),
        nn.Sigmoid()
    )
  def forward(self , features):
    out = self.network(features)
    return out

In [ ]:
loss_function = nn.BCELoss()

In [ ]:
epochs = 25
learning_rate = 0.1

In [ ]:
model = MySimpleNN(X_train_tensor.shape[1])
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# for epoch in range(epochs):
#   y_pred = model(X_train_tensor)
#   loss = loss_function(y_pred , y_train_tensor.unsqueeze(1)) # Add unsqueeze to match dimensions
#   loss.backward()
#   optimizer.step()
#   optimizer.zero_grad()

#   if (epoch + 1) % 10 == 0:
#     print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

## DATALOADER DATASET

In [ ]:
from torch.utils.data import Dataset , DataLoader

In [ ]:
class CustomDataset(Dataset):
  def __init__(self , features , labels):
    self.features = features;
    self.labels = labels
  def __len__(self):
    return len(self.features)
  def __getitem__(self , idx):
    return self.features[idx] , self.labels[idx]


In [ ]:
train_dataset = CustomDataset(X_train_tensor , y_train_tensor)
test_dataset = CustomDataset(X_test_tensor , y_test_tensor)

In [ ]:
train_loader = DataLoader(train_dataset , batch_size=32 , shuffle=True)
test_loader = DataLoader(test_dataset , batch_size=32 , shuffle=False)

## Training Pipeline

In [ ]:
for epoch in range(epochs):
  for batch_features , batch_labels in train_loader:
    y_pred = model(batch_features)
    loss = loss_function(y_pred , batch_labels.unsqueeze(1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f"Epoch: {epoch+1} , Loss: {loss.item()}")

Epoch: 1 , Loss: 0.013427083380520344
Epoch: 2 , Loss: 0.019291019067168236
Epoch: 3 , Loss: 0.009634907357394695
Epoch: 4 , Loss: 0.079347625374794
Epoch: 5 , Loss: 0.08152525871992111
Epoch: 6 , Loss: 0.1413227617740631
Epoch: 7 , Loss: 0.021135738119482994
Epoch: 8 , Loss: 0.006870943121612072
Epoch: 9 , Loss: 0.02406330220401287
Epoch: 10 , Loss: 0.04929933696985245
Epoch: 11 , Loss: 0.0024553989060223103
Epoch: 12 , Loss: 0.0011660745367407799
Epoch: 13 , Loss: 0.007129820995032787
Epoch: 14 , Loss: 0.028507951647043228
Epoch: 15 , Loss: 0.023820042610168457
Epoch: 16 , Loss: 0.010378402657806873
Epoch: 17 , Loss: 0.056941919028759
Epoch: 18 , Loss: 0.0660540834069252
Epoch: 19 , Loss: 0.012175110168755054
Epoch: 20 , Loss: 0.0007235506200231612
Epoch: 21 , Loss: 0.06702228635549545
Epoch: 22 , Loss: 0.0022499519400298595
Epoch: 23 , Loss: 0.13384534418582916
Epoch: 24 , Loss: 0.02406352199614048
Epoch: 25 , Loss: 0.04998550936579704


In [ ]:
model.eval()
accuracy_list = []

with torch.no_grad():
  for batch_features , batch_labels in test_loader:
    y_pred = model(batch_features)
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred == batch_labels.unsqueeze(1)).float().mean()
    accuracy_list.append(accuracy.item())

print(f"Accuracy: {np.mean(accuracy_list)}")

Accuracy: 0.9921875


## NEURAL NETWORK USING ANN WITH FASHION MNIST

In [2]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset , DataLoader
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torch.optim as optim
import matplotlib.pyplot as plt

In [5]:
torch.manual_seed(42)

df = pd.read_csv('fashion-mnist_train.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0,0,0,0,0,0,0,5,0,...,0.0,0.0,0.0,30.0,43.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,1,2,0,0,0,0,...,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df.shape

(9000, 785)

In [7]:
X = df.iloc[: , 1:].values
y = df.iloc[: , 0].values

X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , random_state=42)

In [8]:
X_train = X_train/255.0
X_test = X_test/255.0

In [9]:
class CustomDataset(Dataset):
  def __init__(self , features , labels):
    self.features = torch.tensor(features , dtype=torch.float32)
    self.label = torch.tensor(labels , dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self , idx):
    return self.features[idx] , self.label[idx]

In [10]:
train_dataset = CustomDataset(X_train , y_train)
test_dataset = CustomDataset(X_test , y_test)

In [11]:
train_loader = DataLoader(train_dataset , batch_size=32 , shuffle=True , pin_memory=True)
test_loader = DataLoader(test_dataset , batch_size=32 , shuffle=False , pin_memory= True)

In [12]:
class MyNN(nn.Module):
  def __init__(self , features):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(features , 256),
        nn.ReLU(),
        nn.Linear(256 , 128),
        nn.ReLU(),
        nn.Linear(128 , 64),
        nn.ReLU(),
        nn.Linear(64 , 10)
    )
  def forward(self , x):
    return self.model(x)

In [13]:
learning_rate=0.1
epochs = 100


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [15]:
model = MyNN(X_train.shape[1])
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters() , lr=learning_rate)

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters() , lr=learning_rate)

for epoch in range(epochs):
  total_epoch_loss = 0.0
  for batch_feature , batch_labels in train_loader:
    batch_feature, batch_labels = batch_feature.to(device), batch_labels.to(device)
    outputs = model(batch_feature)
    loss = loss_fn(outputs , batch_labels.long())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch : {epoch+1} , Loss : {avg_loss}')

Epoch : 1 , Loss : 1.4541950358284845
Epoch : 2 , Loss : 0.7490363283952077
Epoch : 3 , Loss : 0.6174940268860923
Epoch : 4 , Loss : 0.5260173992315929
Epoch : 5 , Loss : 0.4854732659790251
Epoch : 6 , Loss : 0.44676936897966596
Epoch : 7 , Loss : 0.4150928458240297
Epoch : 8 , Loss : 0.3974886541234122
Epoch : 9 , Loss : 0.3872898763087061
Epoch : 10 , Loss : 0.3658308015598191
Epoch : 11 , Loss : 0.3461954251594014
Epoch : 12 , Loss : 0.3243234274453587
Epoch : 13 , Loss : 0.3244156949056519
Epoch : 14 , Loss : 0.2989657473233011
Epoch : 15 , Loss : 0.30048429267274007
Epoch : 16 , Loss : 0.2845891111426883
Epoch : 17 , Loss : 0.28081654930280314
Epoch : 18 , Loss : 0.2613327579365836
Epoch : 19 , Loss : 0.24409287406338587
Epoch : 20 , Loss : 0.24433016318413947
Epoch : 21 , Loss : 0.2336761440171136
Epoch : 22 , Loss : 0.23393872747818628
Epoch : 23 , Loss : 0.22438455553518402
Epoch : 24 , Loss : 0.21079221942358548
Epoch : 25 , Loss : 0.20856203423606026
Epoch : 26 , Loss : 0.226

In [18]:
model.eval()

with torch.no_grad():
  total = 0
  correct = 0

  with torch.no_grad():



    for batch_feature , batch_labels in test_loader:
      batch_feature, batch_labels = batch_feature.to(device), batch_labels.to(device)
      outputs = model(batch_feature)

      predicted = torch.max(outputs , 1)

      total += batch_labels.size(0)
      correct += (predicted.indices == batch_labels).sum().item()

  accuracy = 100 * correct / total

In [19]:
accuracy

85.5